In [1]:
import networkx as nx
import pandas as pd
from twitter_network.nodes.setup import create_sample, preprocess_sample
import pickle

In [69]:
%reload_kedro
real_50 = io.load('train_50_hidden')
fake_50 = io.load('train_50_fake')
train_100 = io.load('train_100')
train_300 = io.load('train_300')
train_600 = io.load('train_600')
test = io.load('edges_test')

2019-07-01 23:06:26,607 - INFO - ** Kedro project Twitter Network
2019-07-01 23:06:26,610 - INFO - Loading: /Users/tomduke/Documents/Academic/MBusA/Modules/M3 - Advanced Analytics/Machine Learning/twitter-network/conf/base/logging.yml
2019-07-01 23:06:26,622 - INFO - Loading: /Users/tomduke/Documents/Academic/MBusA/Modules/M3 - Advanced Analytics/Machine Learning/twitter-network/conf/base/catalog.yml
2019-07-01 23:06:26,653 - INFO - Loading: /Users/tomduke/Documents/Academic/MBusA/Modules/M3 - Advanced Analytics/Machine Learning/twitter-network/conf/base/credentials.yml
2019-07-01 23:06:26,658 - INFO - Loading: /Users/tomduke/Documents/Academic/MBusA/Modules/M3 - Advanced Analytics/Machine Learning/twitter-network/conf/base/parameters.yml
2019-07-01 23:06:26,678 - INFO - Defined global variables proj_dir, proj_name, conf and io
2019-07-01 23:06:26,680 - INFO - Loading data from `train_50_hidden` (PickleLocalDataSet)...
2019-07-01 23:06:26,812 - INFO - Loading data from `train_50_fake` 

In [70]:
train_small = create_sample(hidden = real_50, fakes = fake_50)
train_small.head()

,edge,label
0,"(3514248, 214369)",0
1,"(2898003, 4793264)",1
2,"(2771452, 173634)",1
3,"(2696527, 3204376)",1
4,"(1470184, 3559592)",0


In [71]:
print('Number of duplicates in small sample: {}'.format(sum(train_small.duplicated())))

Number of duplicates in small sample: 0


In [72]:
train_big = preprocess_sample(pd.concat([train_100, train_300, train_600]), test = test); train_big.head()

2019-07-01 23:08:07,856 - WARNING - Dropping 122 test edge(s) from sample.
2019-07-01 23:08:08,532 - WARNING - Dropping 221177 duplicate(s) from sample.
2019-07-01 23:08:09,097 - WARNING - Dropping 438 self-loop(s) from sample.


,edge,label
271449,"(381694, 2742770)",1
59502,"(2606444, 1486892)",1
41931,"(1202094, 662501)",1
405792,"(3769409, 1481223)",0
82802,"(1108399, 689315)",1


In [73]:
print('Number of duplicates in big sample: {}'.format(sum(train_big.duplicated())))

Number of duplicates in big sample: 0


In [74]:
# Concatenate big and small samples and remove duplicates
train_master = pd.concat([train_big, train_small]).drop_duplicates(); train_master.head()

,edge,label
271449,"(381694, 2742770)",1
59502,"(2606444, 1486892)",1
41931,"(1202094, 662501)",1
405792,"(3769409, 1481223)",0
82802,"(1108399, 689315)",1


In [75]:
print('Number of duplicates in master sample: {}'.format(sum(train_master.duplicated())))

Number of duplicates in master sample: 0


In [76]:
# Reindex dataframe 
train_master = train_master.set_index(pd.Series(range(len(train_master)))); train_master.head()

,edge,label
0,"(381694, 2742770)",1
1,"(2606444, 1486892)",1
2,"(1202094, 662501)",1
3,"(3769409, 1481223)",0
4,"(1108399, 689315)",1


In [77]:
# Randomise the training set
train_master = train_master.sample(frac = 1)

In [78]:
train_master.head(n = 10)

,edge,label
725107,"(2883988, 4839812)",0
200269,"(4070336, 1081184)",1
684733,"(467667, 542853)",1
701077,"(4583130, 4834729)",1
785215,"(4155530, 651507)",0
149432,"(144060, 753157)",0
410577,"(2461372, 2410794)",1
846317,"(685366, 736573)",0
279633,"(1817011, 4329524)",1
793816,"(4493843, 4384804)",0


In [79]:
FILEPATH = "data/02_intermediate/samples/train_master.pkl"

In [80]:
with open(FILEPATH, "wb") as f:
    pickle.dump(edges, f)

In [81]:
with open(FILEPATH, "rb") as f:
    df = pickle.load(f)

df.describe()

,label
count,866888.000000
mean,0.564001
std,0.495887
min,0.000000
25%,0.000000
50%,1.000000
75%,1.000000
max,1.000000


In [89]:
print('Validation size:', len(df) * 0.8)

Validation size: 693510.4
